In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

#Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
import datetime
import pytz
from scipy import stats
from sklearn import linear_model, metrics, ensemble, model_selection,preprocessing

#Settings
pd.set_option("display.max_columns", None)

In [ ]:
#Read train and store csv, join them into single dataframe and print head
df_2017_1 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING.csv', low_memory=False)
df_2017_2 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 2.csv', low_memory=False)
df_2017_3 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 3.csv', low_memory=False)
df_2017_4 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 4.csv',  low_memory=False)
df_2017_5 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 5.csv',  low_memory=False)
df_2017_6 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 6.csv', low_memory=False)
df_2017_7 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 7.csv', low_memory=False)
df_2017_8 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 8.csv', low_memory=False)
df_2017_9 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 9.csv',  low_memory=False)
df_2017_10 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 10.csv', low_memory=False)
df_2017_11 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 11.csv',  low_memory=False)
df_2017_12 = pd.read_csv('Yearly_Data/2017/440927628_T_ONTIME_REPORTING 12.csv', low_memory=False)
result_2017 = pd.concat([df_2017_1,df_2017_2,df_2017_3,df_2017_4,df_2017_5,df_2017_6,df_2017_7,df_2017_8,df_2017_9,df_2017_10,df_2017_11,df_2017_12])

FileNotFoundError: ignored

In [ ]:
#Read train and store csv, join them into single dataframe and print head
df_2018_1 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING.csv', low_memory=False)
df_2018_2 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 2.csv', low_memory=False)
df_2018_3 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 3.csv', low_memory=False)
df_2018_4 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 4.csv',  low_memory=False)
df_2018_5 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 5.csv',  low_memory=False)
df_2018_6 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 6.csv', low_memory=False)
df_2018_7 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 7.csv', low_memory=False)
df_2018_8 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 8.csv', low_memory=False)
df_2018_9 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 9.csv',  low_memory=False)
df_2018_10 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 10.csv', low_memory=False)
df_2018_11 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 11.csv',  low_memory=False)
df_2018_12 = pd.read_csv('Yearly_Data/2018/440927628_T_ONTIME_REPORTING 12.csv', low_memory=False)
result_2018 = pd.concat([df_2018_1,df_2018_2,df_2018_3,df_2018_4,df_2018_5,df_2018_6,df_2018_7,df_2018_8,df_2018_9,df_2018_10,df_2018_11,df_2018_12])

In [ ]:
# Concatinating data for 2017 and 2018, and creating a safe copy of df
df = pd.concat([result_2017,result_2018])
# Removing rows where arr and dep delay is NAN
df = df[(df['ARR_DELAY'].notna()) & (df['DEP_DELAY'].notna())]

# We don't need cancelled flights in our dataset
df = df[df.CANCELLED !=1]

# Converting flight date to date time
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

# Removing unused columns
variables_to_remove = ['OP_UNIQUE_CARRIER','CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 42','CANCELLED','DIVERTED','FLIGHTS','CANCELLATION_CODE']
df = df.drop(columns=variables_to_remove)

In [ ]:
# Function to convert 24hrs time format to HH:MM
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

# Converting time to HH:MM
df['CRS_ARR_TIME'] = df['CRS_ARR_TIME'].apply(format_heure)
df['ARR_TIME'] = df['ARR_TIME'].apply(format_heure)
df['CRS_DEP_TIME'] = df['CRS_DEP_TIME'].apply(format_heure)
df['DEP_TIME'] = df['DEP_TIME'].apply(format_heure)

In [ ]:
# Top 20 businest airports from wikipedia and filtering the dataframe
busiest_airports_iata = ["ATL", "DFW", "DEN", "ORD", "LAX", "CLT", "LAS", "PHX", "MCO", "SEA", "MIA", "IAH", "JFK", "FLL", "SFO", "EWR", "MSP", "DTW", "BOS", "PHL"]
df = df[df.DEST.isin(busiest_airports_iata) & (df.ORIGIN.isin(busiest_airports_iata))]

In [ ]:
#Weather column headers
columns = ["key", "date", "class", "expire_time_gmt", "obs_id", "obs_name", "valid_time_gmt", "day_ind", "temp", "wx_icon", "icon_extd", "wx_phrase", "pressure_tend", "pressure_desc", "dewPt", "heat_index", "rh", "pressure", "vis", "wc", "wdir", "wdir_cardinal", "gust", "wspd", "max_temp", "min_temp", "precip_total", "precip_hrly", "snow_hrly", "uv_desc", "feels_like", "uv_index", "qualifier", "qualifier_svrty", "blunt_phrase", "terse_phrase", "clds", "water_temp", "primary_wave_period", "primary_wave_height", "primary_swell_period", "primary_swell_height", "primary_swell_direction", "secondary_swell_period", "secondary_swell_height", "secondary_swell_direction",]

# Reading weather data from csv files
path = "Weather_Data/"  # use your path
lst = []

for airport in busiest_airports_iata:
    lst.append(pd.read_csv(path + airport + ".csv", low_memory=False,names = columns))
main_df = pd.concat(lst)    

In [ ]:
# Removing header columns
main_df = main_df[main_df['key'] != 'key']

# Selecting important features, filtering df, and dropping na values
req_cols=['key','date','temp','pressure','wc','wspd', 'wx_phrase','dewPt','vis']
main_df = main_df[req_cols]
main_df = main_df.dropna()

# Removing extra character from airport code that was part of downloaded data, and rest index
main_df['key'] = main_df['key'].str.slice(start=1)
main_df.reset_index(inplace=True)
main_df=main_df.drop(columns=["index"])

In [ ]:
dic = {'ATL': 'America/New_York', 'DFW': 'America/Chicago', 'DEN': 'America/Denver', 'ORD': 'America/Chicago', 'LAX': 'America/Los_Angeles', 'CLT': 'America/New_York', 'LAS': 'America/Los_Angeles', 'PHX': 'America/Phoenix', 'MCO': 'America/New_York',
       'SEA': 'America/Los_Angeles', 'MIA' : 'America/New_York', 'IAH': 'America/Chicago', 'JFK': 'America/New_York', 'FLL': 'America/New_York', 'SFO': 'America/Los_Angeles', 'EWR': 'America/New_York', 'MSP': 'America/Chicago', 'DTW': 'America/Detroit',
       'BOS': 'America/New_York', 'PHL': 'America/New_York'}

In [ ]:
main_df_preprocess = main_df.copy()

In [ ]:
main_df_preprocess['date'] =  pd.to_datetime(main_df_preprocess['date'])
main_df_preprocess['date'] = main_df_preprocess.apply(lambda x: \
                                                      pytz.timezone(dic[x['key']]).localize(x['date']).astimezone(dic[x['key']]),axis=1)
main_df_preprocess['Merge_Date'] = main_df_preprocess['date'].apply(lambda x : x.replace(tzinfo=None))
main_df_preprocess['Merge_Airport'] = main_df_preprocess['key']

# Preparing flights data to merge on departure time
df['Merge_Date'] = df.apply(lambda x: pd.Timestamp(x.FL_DATE).replace(hour=x['CRS_DEP_TIME'].hour, 
                                        minute=x['CRS_DEP_TIME'].minute, second=x['CRS_DEP_TIME'].second), axis=1)
df['Merge_Airport'] = df['ORIGIN']

# Merging flights with weather on departure time
merged_df = pd.merge_asof(df.sort_values('Merge_Date'), main_df_preprocess.sort_values('Merge_Date'),
                          on = "Merge_Date", by='Merge_Airport', direction="nearest", allow_exact_matches=True)

In [ ]:
# Removing final columns which are not needed anymore
remove_final_cols = ['QUARTER', 'DAY_OF_WEEK', 'FL_DATE', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', \
                     'DEST_AIRPORT_ID', 'DEST_CITY_NAME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15', 'ARR_TIME', \
                     'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME',\
                     'ORIGIN_STATE_ABR','DEST_STATE_ABR','Merge_Airport','key','wx_phrase','ARR_DELAY_NEW',\
                     'TAIL_NUM','OP_CARRIER_FL_NUM','Merge_Date','CRS_ARR_TIME','DEST','date']

In [ ]:
final_merged_df = merged_df.drop(columns = remove_final_cols)

In [ ]:
final_merged_df['temp'] = final_merged_df['temp'].astype(float)
final_merged_df['pressure'] = final_merged_df['pressure'].astype(float)
final_merged_df['wc'] = final_merged_df['wc'].astype(float)
final_merged_df['wspd'] = final_merged_df['wspd'].astype(float)
final_merged_df['dewPt'] = final_merged_df['dewPt'].astype(float)
final_merged_df['vis'] = final_merged_df['vis'].astype(float)


In [ ]:
final_merged_df = final_merged_df[final_merged_df.DEP_DELAY_NEW <= 1440]
final_merged_df['crs_dep_hour'] = final_merged_df.apply(lambda x: x['CRS_DEP_TIME'].hour,axis=1)
final_merged_df['crs_dep_minute'] = final_merged_df.apply(lambda x: x['CRS_DEP_TIME'].minute,axis=1)
final_merged_df.drop(columns=['CRS_DEP_TIME'], inplace=True)

In [ ]:
def column_types(df):
    category_cols = []
    for col in df.columns:
        if df[col].dtype.name == 'object':
            category_cols.append(col)
    return category_cols
category_cols = column_types(final_merged_df)
category_cols

le1 = preprocessing.LabelEncoder()
le1.fit(final_merged_df['OP_CARRIER'].astype(str))
final_merged_df['OP_CARRIER'] = le1.transform(final_merged_df['OP_CARRIER'])

le2 = preprocessing.LabelEncoder()
le2.fit(final_merged_df['ORIGIN'].astype(str))
final_merged_df['ORIGIN'] = le2.transform(final_merged_df['ORIGIN'])

In [ ]:
train, test = model_selection.train_test_split(final_merged_df, test_size=0.2, random_state = 42)
train_x = np.array(train.drop(['DEP_DELAY_NEW'], axis=1))
x_label = np.array(train['DEP_DELAY_NEW'])
test_y = np.array(test.drop(['DEP_DELAY_NEW'], axis = 1))
y_label = np.array(test['DEP_DELAY_NEW'])

In [ ]:
import xgboost as xg
xgb = xg.XGBRegressor(learning_rate=0.2, n_estimators = 10, random_state = 42)
model_xgb = xgb.fit(train_x, x_label)

In [ ]:
y_pred = model_xgb.predict(test_y)
rmse_1 = np.sqrt(metrics.mean_absolute_error(y_label, y_pred))
rmse_1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rdr = RandomForestRegressor(n_estimators=200, criterion='mse', n_jobs=50, random_state=50)
rdr_model = rdr.fit(train_x, x_label)

In [ ]:
y_pred = rdr_model.predict(test_y)

In [ ]:
rmse_1 = np.sqrt(metrics.mean_absolute_error(y_label, y_pred))
rmse_1